In [1]:
# generate links / json states programatically by cell type or anything, probably hack-y but it works
from caveclient import CAVEclient
import datetime
import pandas as pd
import numpy as np
import json
import xlwt
from xlwt import Workbook, Formula

client = CAVEclient()
datastack_name = 'fanc_production_mar2021'
client = CAVEclient(datastack_name)
client.materialize.version = 604
timestamp = datetime.datetime.fromtimestamp(1685002201.254304)

In [2]:
# toggle

ngl_base = 'https://neuromancer-seung-import.appspot.com/' 


In [3]:
def make_json(seg_ids, hidden_ids):
    state = {
    "layers": [
      {
        "source": "precomputed://gs://lee-lab_female-adult-nerve-cord/alignmentV4/em/rechunked",
        "type": "image",
        "blend": "default",
        "shaderControls": {},
        "name": "FANC EM"
      },
      {
        "type": "segmentation",
        "mesh": "precomputed://gs://lee-lab_female-adult-nerve-cord/meshes/FANC/FANC_neurons/meshes",
        "selectedAlpha": 0.4,
        "colorSeed": 3788587020,
        "segments": seg_ids,
        "hiddenSegments": hidden_ids,
        "skeletonRendering": {
          "mode2d": "lines_and_points",
          "mode3d": "lines"
        },
        "name": "published FANC neurons"
      },
      {
        "type": "segmentation",
        "mesh": "precomputed://gs://lee-lab_female-adult-nerve-cord/alignmentV4/volume_meshes/meshes",
        "objectAlpha": 0.1,
        "hideSegmentZero": False,
        "ignoreSegmentInteractions": True,
        "segmentColors": {
          "1": "#bfbfbf",
          "2": "#d343d6"
        },
        "segments": [
          "1",
          "2"
        ],
        "skeletonRendering": {
          "mode2d": "lines_and_points",
          "mode3d": "lines"
        },
        "name": "region outlines"
      }
    ],
    "navigation": {
      "pose": {
        "position": {
          "voxelSize": [
            4.300000190734863,
            4.300000190734863,
            45
          ],
          "voxelCoordinates": [38734, 148426, 2200
          ]
        }
      },
      "zoomFactor": 12
    },
    "showAxisLines": False,
    "showDefaultAnnotations": False,
    "perspectiveZoom": 6062.41070084089,
    "showSlices": False,
    "gpuMemoryLimit": 4000000000,
    "systemMemoryLimit": 4000000000,
    "concurrentDownloads": 64,
    "jsonStateServer": "https://global.daf-apis.com/nglstate/api/v1/post",
    "selectedLayer": {
      "layer": "published FANC neurons",
      "visible": True
    },
    "layout": "3d"
  }

    return state

In [4]:
leg_df = pd.read_pickle('dfs_pre_to_mn/pre_to_mn_df_matched_typed_with_nt_v604_20230525.pkl')
leg_mns = leg_df.columns.get_level_values('segID').to_list()

wing_df = pd.read_pickle('preMN_to_MN_wing_v604_multi_df.pkl')
wing_mn_df = pd.read_pickle('wingMN_properties_v604.pkl')
wing_mns = wing_mn_df.MN_id.to_list()


In [19]:
leg_dict = {
    "coxa_swing" : [648518346491659326,648518346479325585,648518346517827432,648518346486895331,648518346487504531,648518346517437482,648518346487756866],
    "coxa_stance" : [648518346478992848,648518346488854257,648518346490902957,648518346489802071,648518346479077839,648518346489076201],
    "trochanter_extend" : [648518346504856435,648518346496195084,648518346482347156,648518346492578233,648518346515708666,648518346467061998,648518346502537286,648518346498089882],
    "trochanter_flex" : [648518346487577002,648518346514275550,648518346517824616,648518346496812333,648518346479840662,648518346499918803,648518346514277931,648518346488873565,648518346498498324,648518346490899373,648518346489470041],
    "femur_reductor" : [648518346487743188,648518346493238592,648518346496372087,648518346502536518,648518346492338559,648518346494195703],
    "tibia_extend" : [648518346493238080,648518346495797355],
    "tibia_flex_A" : [648518346496059082,648518346494196727,648518346496932836,648518346484809885,648518346493836024,648518346486906595,648518346482926501,648518346489731855,648518346494269831,648518346499176513],
    "tibia_flex_B" : [648518346490901677,648518346496057290,648518346493202674,648518346479331004,648518346482925989,648518346517830760,648518346495467855],
    "tibia_flex_C" : [648518346496852276,648518346491041571],
    "substrate_grip" : [648518346491158561,648518346475434081,648518346504867443,648518346486902499,648518346484620291,648518346515711482,648518346493203442,648518346496934372],
    "tarsus_control" : [648518346486890393,648518346491158817]
}

wing_dict = {
    "DLM" : [648518346472269618,648518346486563887,648518346515087943,648518346494250631,648518346486629017],
    "DVM" : [648518346492294527,648518346503730081,648518346488827114,648518346512153646,648518346492384195,648518346499049025,648518346509427075],
    "tension" : [648518346492244419,648518346497125761,648518346490085915,648518346486682876,648518346472953700],
    "steerA" : [648518346488622926,648518346487480490,648518346509604297],
    "steerB" : [648518346470491902,648518346471876251,648518346499256923],
    "steerC" : [648518346493634040,648518346501373528],
    "steerhg2" : [648518346492048172],
    "steerD" : [648518346482131604,648518346480882144,648518346515574010,648518346492294783]
}


In [32]:
# update leg_df module names to match dict
# energy.rename(index={'Republic of Korea':'South Korea'},inplace=True)

leg_df.rename(index = {'thorax_swing': 'coxa_swing', 'thorax_stance': 'coxa_stance', 'trochanter_extension' : 'trochanter_extend', 
                         'trochanter_flexion' : 'trochanter_flex', 'tibia_extensor' : 'tibia_extend', 'main_tibia_flexor' : 'tibia_flex_A',
                         'auxiliary_tibia_flexor_B' : 'tibia_flex_B', 'auxiliary_tibia_flexor_E' : 'tibia_flex_C', 'ltm' : 'substrate_grip',
                         'tarsus_depressor_med_venU' : 'tarsus_control', 'tarsus_depressor_noid': 'tarsus_control'}, inplace=True)

In [121]:
# leg cell type update
leg_df.rename(index = {'RVD' : '8A','Rcore_' : '8A', 'RCore' : '8A', 'R6' : '8A', 'R5' : '8A', 'R4' : '8A', 'R3' : '8A', 'R2' : '8A', '8A- Rind' : '8A',
                        'R1' : '8A', 'R0' : '8A', '8A+' : '8A', '8A-' : '8A', '8A_' : '8A', '14A_' : '14A', '6B_' : '6B', '15B' : 'LinE',
                        '16B' : 'hlx16', '16B_' : 'hlx16', '16B_small' : 'hlx16', '8A? 16B?' : 'hlx16', '4B_' : '4B', '4B-' : '4B', 
                        '4B+' : '4B', '22A_' : '22A', '22A?' : '22A', '21A_' : '21A', '9Ac_' : '9A', '1B_' : '1B', '13B_' : '13B'}, inplace=True)

# merged_premotor = merged_premotor.replace(['RVD','Rcore_', 'RCore', 'R6', 'R5', 'R4', 'R3', 'R2', 'R1', 'R0', '8A+', '8A-', '8A_'], '8A')
# merged_premotor = merged_premotor.replace(['14A_'], '14A')
# merged_premotor = merged_premotor.replace(['6B_'], '6B')
# merged_premotor = merged_premotor.replace(['hlx16', '16B_', '16B_small', '8A? 16B?'], '16B')
# merged_premotor = merged_premotor.replace(['4B_', '4B-', '4B+'], '4B')
# merged_premotor = merged_premotor.replace(['22A_', '22A?'], '22A')
# merged_premotor = merged_premotor.replace(['21A_'], '21A')
# merged_premotor = merged_premotor.replace(['9Ac_'], '9A')
# merged_premotor = merged_premotor.replace(['1B_'], '1B')
# merged_premotor = merged_premotor.replace(['13B_'], '13B')

In [122]:
idx = pd.IndexSlice
wing_df_loc = wing_df.loc[idx[:,:,:,['local','local_intersegmental'],:]]
leg_df_loc = leg_df.loc[idx[['local','intersegmental'],:,:,:,:,:]]

In [123]:
leg_preMNs = pd.DataFrame(leg_df_loc.index.get_level_values('segID'))
types = pd.DataFrame(leg_df_loc.index.get_level_values('cell_type'))

leg_preMNs['cell_type'] = types


In [124]:
leg_preMNs = leg_preMNs.rename(columns={'segID':'pre_pt_root_id'})

In [126]:
merged_premotor_df = wing_df_loc.reset_index()[['pre_pt_root_id','cell_type']].merge(leg_preMNs, how = 'outer')
len(merged_premotor_df)

2109

In [135]:
# hemilineage links

wb = Workbook()
sheet1 = wb.add_sheet('hemilineages')
json_base = 'https://neuromancer-seung-import.appspot.com/?json_url=https://raw.githubusercontent.com/tuthill-lab/Lesser_Azevedo_2023/main/jsons/'
cell_types = merged_premotor_df.cell_type.unique()
hidden_ids = []
r = 0
for hemilineage in cell_types:
        seg_ids = merged_premotor_df[merged_premotor_df.cell_type.isin([hemilineage])].pre_pt_root_id.to_list()
        formatted_ids = [str(x) for x in seg_ids]
        state = make_json(seg_ids = formatted_ids,
                          hidden_ids = hidden_ids)
        json_title = str(hemilineage)

        # save a json
        with open('jsons/{}.json'.format(json_title), "w") as outfile:
                json.dump(state, outfile)

        table_link = json_base + json_title
        hyperlink = 'HYPERLINK("{}.json";"{}")'.format(table_link, json_title)
        sheet1.write(r, 1, Formula(hyperlink))
        r = r+1

        wb.save('hemilineage_links.xls')
        

In [ ]:

# make json links and a table for neuroglancer states by cell class and preferred module

# toggle for leg vs wing
# hidden_ids = leg_mns # wing_mns leg_mns
limb_dict = leg_dict # wing_dict leg_dict
df = leg_df # wing_df leg_df
column_title = 'Leg preMNs' # Wing Leg

idx = pd.IndexSlice
# cell_classes = wing_df.index.get_level_values('cell_class').unique().to_list()
cell_classes = leg_df.index.get_level_values('cell_class').unique().to_list()

wb = Workbook()
sheet1 = wb.add_sheet('PreMNs')
sheet1.write(0,1, column_title)
json_base = 'https://neuromancer-seung-import.appspot.com/?json_url=https://raw.githubusercontent.com/tuthill-lab/Lesser_Azevedo_2023/main/jsons/'
r = 0 # table row
for module in limb_dict: # preferred module loop
    r = r+1
    c = 1
    for cell_class in cell_classes: # cell class loop   
        c = c+1 
        json_title = '{}_{}'.format(cell_class,module)
        # module_preMN = module + '_syn' #### WING ONLY

        # seg_ids = wing_df.loc[idx[:,module_preMN,:,cell_class,:]].reset_index().pre_pt_root_id.to_list()
        seg_ids = leg_df.loc[idx[cell_class,module,:,:,:,:]].reset_index().segID.to_list()

    
        formatted_ids = [str(x) for x in seg_ids]
        
        state = make_json(seg_ids = formatted_ids)
        
        # # save a json
        # with open('jsons/{}.json'.format(json_title), "w") as outfile:
        #     json.dump(state, outfile)

        table_link = json_base + json_title
        hyperlink = 'HYPERLINK("{}.json";"{}")'.format(table_link, json_title)
        # '"{} " & HYPERLINK("{}")'.format(json_title[:-5], table_link)
        sheet1.write(r, c, Formula(hyperlink))

wb.save('leg_preMNlinks.xls')
        


In [ ]:
# trouble shooting etc below

In [44]:
hyperlink


'HYPERLINK("https://neuromancer-seung-import.appspot.com/?json_url=https://raw.githubusercontent.com/tuthill-lab/Lesser_Azevedo_2023/main/jsons/local_tarsus_control.json";"local_tarsus_co")'

In [ ]:
# https://neuromancer-seung-import.appspot.com/?json_url=https://raw.githubusercontent.com/tuthill-lab/Lesser_Azevedo_2023/main/jsons/11A.json

In [83]:
table_link


'https://neuromancer-seung-import.appspot.com/?json_url=https://raw.githubusercontent.com/tuthill-lab/Lesser_Azevedo_2023/main/jsons/local_preMNs_steerD_pref.json'

In [ ]:
# save a json
with open('jsons/{}.json'.format(json_title), "w") as outfile:
    json.dump(state, outfile)

In [ ]:
# make a link
new_id = client.state.upload_state_json(state)
client.state.build_neuroglancer_url(new_id, ngl_base)

'https://neuromancer-seung-import.appspot.com/?json_url=https://global.daf-apis.com/nglstate/api/v1/5634631871234048'

In [84]:
# make a table of the links
import xlwt
from xlwt import Workbook, Formula

In [ ]:
wb = Workbook()
sheet1 = wb.add_sheet('preMNs by cell class and preferred module')



#### Wing
idx = pd.IndexSlice
df = wing_df.reset_index() # group by cell type

modules = df.preferred_module.unique()
cell_classes = df.cell_class.unique()

for module in modules:
    module_label = str(module)
    module_df = df[df.preferred_module.isin([module_label])]

    for cell in cell_classes:
        cell_class_label = str(cell)
        
        seg_ids = module_df[module_df.cell_class.isin([cell_class_label])].pre_pt_root_id.to_list()
        formatted_ids = [str(x) for x in seg_ids]
    
        statename = 'preferred_module_{}_{}'.format(module_label[:-4],cell_class_label) 
        state = make_json(name = statename,
                          seg_ids = formatted_ids)
        # new_id = client.state.upload_state_json(state)
        # client.state.build_neuroglancer_url(new_id, ngl_base)

    # save json
        with open('jsons/{}.json'.format(statename), "w") as outfile:
            json.dump(state, outfile)





Exception: invalid worksheet name 'preMNs by cell class and preferred module'

In [ ]:
import xlwt
from xlwt import Workbook
  
# Workbook is created
wb = Workbook()
  
# add_sheet is used to create sheet.
sheet1 = wb.add_sheet('Sheet 1')
  
sheet1.write(1, 0, 'ISBT DEHRADUN')
sheet1.write(2, 0, 'SHASTRADHARA')
sheet1.write(3, 0, 'CLEMEN TOWN')
sheet1.write(4, 0, 'RAJPUR ROAD')
sheet1.write(5, 0, 'CLOCK TOWER')
sheet1.write(0, 1, 'ISBT DEHRADUN')
sheet1.write(0, 2, 'SHASTRADHARA')
sheet1.write(0, 3, 'CLEMEN TOWN')
sheet1.write(0, 4, 'RAJPUR ROAD')
sheet1.write(0, 5, 'CLOCK TOWER')
  
wb.save('xlwt example.xls')

In [ ]:
# trouble shooting and old code

In [ ]:
state

{'layers': [{'source': 'precomputed://gs://lee-lab_female-adult-nerve-cord/alignmentV4/em/rechunked',
   'type': 'image',
   'blend': 'default',
   'shaderControls': {},
   'name': 'FANC EM'},
  {'type': 'segmentation',
   'mesh': 'precomputed://gs://lee-lab_female-adult-nerve-cord/meshes/FANC/FANC_neurons/meshes',
   'selectedAlpha': 0.4,
   'colorSeed': 3788587020,
   'segments': ['648518346496059082',
    '648518346494196727',
    '648518346496932836',
    '648518346484809885',
    '648518346493836024',
    '648518346486906595',
    '648518346482926501',
    '648518346489731855',
    '648518346494269831',
    '648518346499176513'],
   'hiddenSegments': ['648518346487756866',
    '648518346491659326',
    '648518346517437482',
    '648518346517827432',
    '648518346479325585',
    '648518346487504531',
    '648518346486895331',
    '648518346479077839',
    '648518346489076201',
    '648518346490902957',
    '648518346489802071',
    '648518346478992848',
    '648518346488854257',
 

In [ ]:
# import dataframe
wing_df = pd.read_pickle('preMN_to_MN_wing_v604_multi_df.pkl')

In [ ]:
# generate a link based on a cell type
# only for local and intersegmental preMNs

idx = pd.IndexSlice
df = wing_df.loc[idx[:,:,:,['local','local_intersegmental'],:]].reset_index() # group by cell type

cell_types = df.cell_type.unique()

for cell_type in cell_types[1:2]:
    cell_type_label = str(cell_type)
    seg_ids = df[df.cell_type.isin([cell_type_label])].pre_pt_root_id.to_list()
    formatted_ids = [str(x) for x in seg_ids]
    
    state = make_json(name = cell_type_label,
                   seg_ids = formatted_ids)
    new_id = client.state.upload_state_json(state)
    client.state.build_neuroglancer_url(new_id, ngl_base)

    # save json 
    # with open('jsons/{}.json'.format(cell_type_label), "w") as outfile:
    #     json.dump(state, outfile)



In [ ]:
client.state.build_neuroglancer_url(new_id, ngl_base)


'https://neuromancer-seung-import.appspot.com/?json_url=https://global.daf-apis.com/nglstate/api/v1/5047254895296512'

In [ ]:
# generate a link based on preferred module

idx = pd.IndexSlice
df = wing_df.reset_index() # group by cell type

modules = df.preferred_module.unique()

for module in modules:
    module_label = str(module)
    seg_ids = df[df.preferred_module.isin([module_label])].pre_pt_root_id.to_list()
    formatted_ids = [str(x) for x in seg_ids]
    
    state = make_json(name = module_label,
                   seg_ids = formatted_ids)
    # new_id = client.state.upload_state_json(state)
    # client.state.build_neuroglancer_url(new_id, ngl_base)

    # save json 
    with open('jsons/{}.json'.format(module_label), "w") as outfile:
        json.dump(state, outfile)


In [ ]:
# jsons per cell class / per preferred module 

idx = pd.IndexSlice
df = wing_df.reset_index() # group by cell type

modules = df.preferred_module.unique()
cell_classes = df.cell_class.unique()

for module in modules:
    module_label = str(module)
    module_df = df[df.preferred_module.isin([module_label])]

    for cell in cell_classes:
        cell_class_label = str(cell)
        
        seg_ids = module_df[module_df.cell_class.isin([cell_class_label])].pre_pt_root_id.to_list()
        formatted_ids = [str(x) for x in seg_ids]
    
        statename = 'preferred_module_{}_{}'.format(module_label[:-4],cell_class_label) 
        state = make_json(name = statename,
                          seg_ids = formatted_ids)
        # new_id = client.state.upload_state_json(state)
        # client.state.build_neuroglancer_url(new_id, ngl_base)

    # save json
        with open('jsons/{}.json'.format(statename), "w") as outfile:
            json.dump(state, outfile)



In [ ]:
# module jsons 
mn_df = pd.read_pickle('wingMN_properties_v604.pkl')

# modules
DLM_mod = ['DLM_1','DLM_2','DLM_3','DLM_4','DLM_5']
DVM_mod = ['DVM_1a','DVM_1b','DVM_1c','DVM_2a','DVM_2b','DVM_3a','DVM_3b']
tension_mod = ['PS1','PSn_u','PS2','dtpmn_u','tp2']
steerA_mod = ['tpn','iii1','iii3']
steerB_mod = ['i2_u','i1','b3_u']
steerC_mod = ['b2','b1']
hg2_mod = ['hg2']
steerD_mod = ['hg1_u','hg3','hg4','iii4a_u']

mn_df.loc[mn_df.MN_label.isin(DLM_mod), 'motor_module'] = 'DLM_module'
mn_df.loc[mn_df.MN_label.isin(DVM_mod), 'motor_module'] = 'DVM_module'
mn_df.loc[mn_df.MN_label.isin(tension_mod), 'motor_module'] = 'tension_module'
mn_df.loc[mn_df.MN_label.isin(steerA_mod), 'motor_module'] = 'steerA_module'
mn_df.loc[mn_df.MN_label.isin(steerB_mod), 'motor_module'] = 'steerB_module'
mn_df.loc[mn_df.MN_label.isin(steerC_mod), 'motor_module'] = 'steerC_module'
mn_df.loc[mn_df.MN_label.isin(hg2_mod), 'motor_module'] = 'hg2_module'
mn_df.loc[mn_df.MN_label.isin(steerD_mod), 'motor_module'] = 'steerD_module'

modules = mn_df.motor_module.unique()

for module in modules:
    module_label = str(module)
    seg_ids = mn_df[mn_df.motor_module.isin([module_label])].MN_id.to_list()
    formatted_ids = [str(x) for x in seg_ids]
    
    state = make_json(name = module_label,
                   seg_ids = formatted_ids)
    # new_id = client.state.upload_state_json(state)
    # client.state.build_neuroglancer_url(new_id, ngl_base)

    # save json 
    with open('jsons/{}_MNs.json'.format(module_label), "w") as outfile:
        json.dump(state, outfile)



In [ ]:
mn_df.loc[mn_df.index.isin(steerD_mod)]

,MN_id,synapses_include_fragments,total_volume_um3,MN_label,motor_module


In [ ]:
pd.read_pickle('wingMN_properties_v604.pkl')

,MN_id,synapses_include_fragments,total_volume_um3,MN_label
0,648518346492244419,11178,4142.678608,PSn_u
1,648518346493634040,9063,5515.440987,b2
2,648518346472269618,8801,8628.326163,DLM_5
3,648518346470491902,8059,5162.048411,i1
4,648518346482131604,7984,3728.941240,iii4a_u
5,648518346471876251,7887,7057.485083,b3_u
6,648518346497125761,7531,3439.709251,PS1
7,648518346480882144,7341,3428.316969,hg4
8,648518346499256923,7248,3980.243544,i2_u
9,648518346486563887,6990,7434.914838,DLM_3


In [ ]:
# if you want a link
client.state.build_neuroglancer_url(new_id, ngl_base)

'https://neuromancer-seung-import.appspot.com/?json_url=https://global.daf-apis.com/nglstate/api/v1/5466062256603136'